In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, layers
from tensorflow.keras.layers import Input, Bidirectional, Dense, Conv1D, Conv3D, LSTM, Flatten, Attention, MultiHeadAttention, GlobalAveragePooling1D, Concatenate, Add, Dropout, Softmax
from tensorflow.keras.regularizers import l2
from utilz import *
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [3]:
visual_clip = load_features('./data/visual_clip.pkl')
acoustic = load_features('./data/acoustic_wav2vec.pkl')
bert_embs = load_features('./data/textual_bert.pkl')
label = load_features('./data/labels.pkl')

# From pretrained

# Model 1

In [4]:
class Attention_Self(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(Attention_Self, self).__init__(**kwargs)
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.S = tf.keras.layers.Dense(1)
        self.units = units

    def call(self, features):
        features_ = tf.expand_dims(features, 1)
        v = self.W1(features)
        q =  self.W2(features_)
        score = tf.nn.tanh(q + v)
        attention_weights = tf.nn.softmax(self.S(score), axis=1)
        ATTN = attention_weights * (v)
        ATTN = tf.reduce_sum(ATTN, axis=1)
        
        return ATTN
        
    def get_config(self):
        config = super(Attention_Self, self).get_config()
        config.update({"units": self.units})
        return config

In [5]:
# './res/V_model_CNNLSTM_clip.tf'
vis_ipt = Input((10, 512))
v_model = tf.keras.models.load_model('./res/V_model_CNNLSTM_clip.tf/')
v_model = Model(inputs=v_model.inputs, outputs=v_model.layers[-2].output)
for l in v_model.layers:
    l.trainable = False
vis_h = v_model(vis_ipt)

# './res/A_model_Att-BLSTM_wav2vec_v2.tf'
aud_ipt = Input((128,512))
a_model = tf.keras.models.load_model('./res/A_model_Att-BLSTM_wav2vec_v2.tf/')
a_model = Model(inputs=a_model.inputs, outputs=a_model.layers[-2].output)
for l in a_model.layers:
    l.trainable = False
aud_h = a_model(aud_ipt)

# './res/T_model_AttCNN_bert.tf'
tex_ipt = Input((36,768))
t_model = tf.keras.models.load_model('./res/T_model_AttCNN_bert.tf/')
t_model = Model(inputs=t_model.inputs, outputs=[t_model.layers[-4].output,t_model.layers[-3].output])
for l in t_model.layers:
    l.trainable = False
tex_q, tex_qv_attention = t_model(tex_ipt)

h = Concatenate()([vis_h, aud_h, tex_q, tex_qv_attention])
h = Dense(64)(h)
res = Dense(3, activation='softmax')(h)

model = Model(inputs=[vis_ipt, aud_ipt, tex_ipt], outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

2022-10-18 12:12:47.170728: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-18 12:12:47.172087: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-18 12:12:47.172237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-18 12:12:47.172313: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 512)]    0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 128, 512)]   0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 36, 768)]    0           []                               
                                                                                                  
 model (Functional)             (None, 64)           147904      ['input_1[0][0]']                
                                                                                            

In [6]:
callback_list = [ModelCheckpoint(filepath='./res/multi_model_concate_pretrained.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=[np.asarray(visual_clip['train']), np.asarray(acoustic['train']), np.asarray(bert_embs['train'])], y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[[np.asarray(visual_clip['valid']), np.asarray(acoustic['valid']), np.asarray(bert_embs['valid'])], np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30


2022-10-18 12:12:53.571905: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500


 6/86 [=>............................] - ETA: 0s - loss: 1.0731 - acc: 0.5625  

2022-10-18 12:12:53.913731: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


84/86 [============================>.] - ETA: 0s - loss: 0.5750 - acc: 0.7701

INFO:tensorflow:Assets written to: ./res/multi_model_concate_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_concate_pretrained.tf/assets


86/86 [==============================] - 11s 101ms/step - loss: 0.5698 - acc: 0.7727 - val_loss: 0.9059 - val_acc: 0.6798
Epoch 2/30
86/86 [==============================] - 1s 11ms/step - loss: 0.3698 - acc: 0.8567 - val_loss: 0.9600 - val_acc: 0.6579
Epoch 3/30
86/86 [==============================] - 1s 15ms/step - loss: 0.3404 - acc: 0.8677 - val_loss: 0.9898 - val_acc: 0.6623
Epoch 4/30
86/86 [==============================] - 1s 14ms/step - loss: 0.3115 - acc: 0.8721 - val_loss: 1.0300 - val_acc: 0.6579
Epoch 5/30
86/86 [==============================] - 1s 13ms/step - loss: 0.2894 - acc: 0.8845 - val_loss: 1.0941 - val_acc: 0.6732
Epoch 6/30
86/86 [==============================] - 1s 13ms/step - loss: 0.2873 - acc: 0.8889 - val_loss: 1.1560 - val_acc: 0.6579
Epoch 7/30
86/86 [==============================] - 1s 13ms/step - loss: 0.2755 - acc: 0.8904 - val_loss: 1.1826 - val_acc: 0.6623
Epoch 8/30
86/86 [==============================] - 1s 12ms/step - loss: 0.2705 - acc: 0.891

In [7]:
model = tf.keras.models.load_model('./res/multi_model_concate_pretrained.tf/')
pred = model.predict([np.asarray(visual_clip['test']), np.asarray(acoustic['test']), np.asarray(bert_embs['test'])])
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

         Neg     0.7109    0.8427    0.7712       248
         Pos     0.6288    0.5929    0.6103       140
         Neu     0.3548    0.1594    0.2200        69

    accuracy                         0.6630       457
   macro avg     0.5648    0.5317    0.5338       457
weighted avg     0.6320    0.6630    0.6387       457

[[209  29  10]
 [ 47  83  10]
 [ 38  20  11]]


# Model 2

In [8]:
# './res/V_model_CNNLSTM_clip.tf'
vis_ipt = Input((10, 512))
v_model = tf.keras.models.load_model('./res/V_model_CNNLSTM_clip.tf/')
v_model = Model(inputs=v_model.inputs, outputs=v_model.layers[-2].output)
for l in v_model.layers:
    l.trainable = False
vis_h = v_model(vis_ipt)

# './res/A_model_Att-BLSTM_wav2vec_v2.tf'
aud_ipt = Input((128,512))
a_model = tf.keras.models.load_model('./res/A_model_Att-BLSTM_wav2vec_v2.tf/')
a_model = Model(inputs=a_model.inputs, outputs=a_model.layers[-2].output)
for l in a_model.layers:
    l.trainable = False
aud_h = a_model(aud_ipt)

# './res/T_model_AttCNN_bert.tf'
tex_ipt = Input((36,768))
t_model = tf.keras.models.load_model('./res/T_model_AttCNN_bert.tf/')
t_model = Model(inputs=t_model.inputs, outputs=[t_model.layers[-4].output,t_model.layers[-3].output])
for l in t_model.layers:
    l.trainable = False
tex_q, tex_qv_attention = t_model(tex_ipt)

h = Concatenate()([vis_h, aud_h, tex_q, tex_qv_attention])
h = Attention_Self(256)(h)
h = Dense(64)(h)
res = Dense(3, activation='softmax')(h)

model = Model(inputs=[vis_ipt, aud_ipt, tex_ipt], outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 10, 512)]    0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 128, 512)]   0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 36, 768)]    0           []                               
                                                                                                  
 model_4 (Functional)           (None, 64)           147904      ['input_4[0][0]']                
                                                                                            

In [9]:
callback_list = [ModelCheckpoint(filepath='./res/multi_model_selfatt_pretrained.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=[np.asarray(visual_clip['train']), np.asarray(acoustic['train']), np.asarray(bert_embs['train'])], y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[[np.asarray(visual_clip['valid']), np.asarray(acoustic['valid']), np.asarray(bert_embs['valid'])], np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
82/86 [===========================>..] - ETA: 0s - loss: 0.9312 - acc: 0.5762

INFO:tensorflow:Assets written to: ./res/multi_model_selfatt_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_selfatt_pretrained.tf/assets


86/86 [==============================] - 11s 108ms/step - loss: 0.9165 - acc: 0.5855 - val_loss: 1.0227 - val_acc: 0.6118
Epoch 2/30
86/86 [==============================] - 1s 14ms/step - loss: 0.6050 - acc: 0.7558 - val_loss: 1.0721 - val_acc: 0.6118
Epoch 3/30
86/86 [==============================] - ETA: 0s - loss: 0.5224 - acc: 0.7939

INFO:tensorflow:Assets written to: ./res/multi_model_selfatt_pretrained.tf/assets


...

INFO:tensorflow:Assets written to: ./res/multi_model_selfatt_pretrained.tf/assets


86/86 [==============================] - 8s 94ms/step - loss: 0.4228 - acc: 0.8341 - val_loss: 0.9791 - val_acc: 0.6469
Epoch 6/30
86/86 [==============================] - 1s 16ms/step - loss: 0.3894 - acc: 0.8545 - val_loss: 1.0413 - val_acc: 0.6294
Epoch 7/30
84/86 [============================>.] - ETA: 0s - loss: 0.3917 - acc: 0.8527

INFO:tensorflow:Assets written to: ./res/multi_model_selfatt_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_selfatt_pretrained.tf/assets


86/86 [==============================] - 9s 100ms/step - loss: 0.3943 - acc: 0.8516 - val_loss: 0.9496 - val_acc: 0.6206
Epoch 8/30
86/86 [==============================] - 1s 11ms/step - loss: 0.3487 - acc: 0.8626 - val_loss: 0.9935 - val_acc: 0.6272
Epoch 9/30
86/86 [==============================] - 1s 12ms/step - loss: 0.3179 - acc: 0.8750 - val_loss: 1.0675 - val_acc: 0.6623
Epoch 10/30
86/86 [==============================] - 1s 11ms/step - loss: 0.3170 - acc: 0.8816 - val_loss: 1.1124 - val_acc: 0.6053
Epoch 11/30
86/86 [==============================] - 1s 12ms/step - loss: 0.3059 - acc: 0.8787 - val_loss: 1.0812 - val_acc: 0.6206
Epoch 12/30
86/86 [==============================] - 1s 12ms/step - loss: 0.3124 - acc: 0.8816 - val_loss: 1.1041 - val_acc: 0.6272
Epoch 13/30
86/86 [==============================] - 1s 12ms/step - loss: 0.2843 - acc: 0.8925 - val_loss: 1.2006 - val_acc: 0.6272
Epoch 14/30
86/86 [==============================] - 1s 11ms/step - loss: 0.2833 - acc: 0

In [10]:
model = tf.keras.models.load_model('./res/multi_model_selfatt_pretrained.tf/')
pred = model.predict([np.asarray(visual_clip['test']), np.asarray(acoustic['test']), np.asarray(bert_embs['test'])])
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

         Neg     0.7319    0.8145    0.7710       248
         Pos     0.5759    0.6500    0.6107       140
         Neu     0.3913    0.1304    0.1957        69

    accuracy                         0.6608       457
   macro avg     0.5664    0.5317    0.5258       457
weighted avg     0.6327    0.6608    0.6350       457

[[202  42   4]
 [ 39  91  10]
 [ 35  25   9]]


In [11]:
# './res/V_model_CNNLSTM_clip.tf'
vis_ipt = Input((10, 512))
v_model = tf.keras.models.load_model('./res/V_model_CNNLSTM_clip.tf/')
v_model = Model(inputs=v_model.inputs, outputs=v_model.layers[-2].output)
for l in v_model.layers:
    l.trainable = False
vis_h = v_model(vis_ipt)

# './res/A_model_Att-BLSTM_wav2vec_v2.tf'
aud_ipt = Input((128,512))
a_model = tf.keras.models.load_model('./res/A_model_Att-BLSTM_wav2vec_v2.tf/')
a_model = Model(inputs=a_model.inputs, outputs=a_model.layers[-2].output)
for l in a_model.layers:
    l.trainable = False
aud_h = a_model(aud_ipt)

# './res/T_model_AttCNN_bert.tf'
tex_ipt = Input((36,768))
t_model = tf.keras.models.load_model('./res/T_model_AttCNN_bert.tf/')
t_model = Model(inputs=t_model.inputs, outputs=[t_model.layers[-4].output,t_model.layers[-3].output])
for l in t_model.layers:
    l.trainable = False
tex_q, tex_qv_attention = t_model(tex_ipt)

h = Concatenate()([vis_h, aud_h, tex_q, tex_qv_attention])
h = Attention(256)([h,h])
h = Dense(64)(h)
res = Dense(3, activation='softmax')(h)

model = Model(inputs=[vis_ipt, aud_ipt, tex_ipt], outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 10, 512)]    0           []                               
                                                                                                  
 input_8 (InputLayer)           [(None, 128, 512)]   0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 36, 768)]    0           []                               
                                                                                                  
 model_8 (Functional)           (None, 64)           147904      ['input_7[0][0]']                
                                                                                           

In [12]:
callback_list = [ModelCheckpoint(filepath='./res/multi_model_selfattv2_pretrained.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=[np.asarray(visual_clip['train']), np.asarray(acoustic['train']), np.asarray(bert_embs['train'])], y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[[np.asarray(visual_clip['valid']), np.asarray(acoustic['valid']), np.asarray(bert_embs['valid'])], np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
82/86 [===========================>..] - ETA: 0s - loss: 0.7108 - acc: 0.7195

INFO:tensorflow:Assets written to: ./res/multi_model_selfattv2_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_selfattv2_pretrained.tf/assets


86/86 [==============================] - 10s 103ms/step - loss: 0.7062 - acc: 0.7208 - val_loss: 0.9437 - val_acc: 0.6162
Epoch 2/30
86/86 [==============================] - 1s 11ms/step - loss: 0.5702 - acc: 0.8041 - val_loss: 1.0280 - val_acc: 0.6491
Epoch 3/30
86/86 [==============================] - 1s 15ms/step - loss: 0.5810 - acc: 0.7931 - val_loss: 0.9602 - val_acc: 0.6228
Epoch 4/30
86/86 [==============================] - 1s 12ms/step - loss: 0.5404 - acc: 0.8048 - val_loss: 1.0127 - val_acc: 0.6250
Epoch 5/30
86/86 [==============================] - 1s 13ms/step - loss: 0.5488 - acc: 0.8099 - val_loss: 1.0040 - val_acc: 0.6316
Epoch 6/30
86/86 [==============================] - 1s 14ms/step - loss: 0.5706 - acc: 0.7946 - val_loss: 1.0112 - val_acc: 0.5987
Epoch 7/30
86/86 [==============================] - 1s 14ms/step - loss: 0.5761 - acc: 0.7931 - val_loss: 1.0332 - val_acc: 0.5921
Epoch 8/30
86/86 [==============================] - 1s 13ms/step - loss: 0.5484 - acc: 0.804

In [13]:
model = tf.keras.models.load_model('./res/multi_model_selfattv2_pretrained.tf/')
pred = model.predict([np.asarray(visual_clip['test']), np.asarray(acoustic['test']), np.asarray(bert_embs['test'])])
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 1s 8ms/step
              precision    recall  f1-score   support

         Neg     0.6906    0.7742    0.7300       248
         Pos     0.5306    0.5571    0.5436       140
         Neu     0.3750    0.1739    0.2376        69

    accuracy                         0.6171       457
   macro avg     0.5321    0.5017    0.5037       457
weighted avg     0.5940    0.6171    0.5986       457

[[192  48   8]
 [ 50  78  12]
 [ 36  21  12]]


# Model 3

In [14]:
class Attention_Cross(tf.keras.layers.Layer):
    
    def __init__(self, units, **kwargs):
        super(Attention_Cross, self).__init__(**kwargs)
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.S = tf.keras.layers.Dense(1)
        self.units = units

    def call(self, features1, features2):
        features2_ = tf.expand_dims(features2, 1)
        v = self.W1(features1)
        # v_ = self.W1(features2)
        q =  self.W2(features2_)
        score = tf.nn.tanh(q + v)
        attention_weights = tf.nn.softmax(self.S(score), axis=1)
        # ATTN = attention_weights * (v+v_)
        ATTN = attention_weights * v
        ATTN = tf.reduce_sum(ATTN, axis=1)
        
#         features2_ = tf.expand_dims(features2, 1)
#         score = tf.nn.tanh(self.W1(features1) + self.W2(features2_))
#         attention_weights = tf.nn.softmax(self.S(score), axis=1)
#         ATTN = attention_weights * (features1 + features2)
#         ATTN = tf.reduce_sum(ATTN, axis=1)
        
        return ATTN
        
    def get_config(self):
        config = super(Attention_Cross, self).get_config()
        config.update({"units": self.units})
        return config


In [15]:
# './res/V_model_CNNLSTM_clip.tf'
vis_ipt = Input((10, 512))
v_model = tf.keras.models.load_model('./res/V_model_CNNLSTM_clip.tf/')
v_model = Model(inputs=v_model.inputs, outputs=v_model.layers[-2].output)
for l in v_model.layers:
    l.trainable = False
vis_h = v_model(vis_ipt)

# './res/A_model_Att-BLSTM_wav2vec_v2.tf'
aud_ipt = Input((128,512))
a_model = tf.keras.models.load_model('./res/A_model_Att-BLSTM_wav2vec_v2.tf/')
a_model = Model(inputs=a_model.inputs, outputs=a_model.layers[-2].output)
for l in a_model.layers:
    l.trainable = False
aud_h = a_model(aud_ipt)

# './res/T_model_AttCNN_bert.tf'
tex_ipt = Input((36,768))
t_model = tf.keras.models.load_model('./res/T_model_AttCNN_bert.tf/')
t_model = Model(inputs=t_model.inputs, outputs=[t_model.layers[-4].output,t_model.layers[-3].output])
for l in t_model.layers:
    l.trainable = False
tex_q, tex_qv_attention = t_model(tex_ipt)

v = Concatenate()([tex_q, tex_qv_attention])
q = Concatenate()([vis_h, aud_h])
h = Attention_Cross(128)(v, q)
# h = Dense(64)(h)
res = Dense(3, activation='softmax')(h)

model = Model(inputs=[vis_ipt, aud_ipt, tex_ipt], outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 36, 768)]    0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, 10, 512)]    0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, 128, 512)]   0           []                               
                                                                                                  
 model_14 (Functional)          [(None, 64),         295040      ['input_12[0][0]']               
                                 (None, 64)]                                               

In [16]:
callback_list = [ModelCheckpoint(filepath='./res/multi_model_crossatt_pretrained.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=[np.asarray(visual_clip['train']), np.asarray(acoustic['train']), np.asarray(bert_embs['train'])], y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[[np.asarray(visual_clip['valid']), np.asarray(acoustic['valid']), np.asarray(bert_embs['valid'])], np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
82/86 [===========================>..] - ETA: 0s - loss: 0.9647 - acc: 0.5473

INFO:tensorflow:Assets written to: ./res/multi_model_crossatt_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_crossatt_pretrained.tf/assets


86/86 [==============================] - 11s 109ms/step - loss: 0.9645 - acc: 0.5482 - val_loss: 0.9768 - val_acc: 0.5680
Epoch 2/30
82/86 [===========================>..] - ETA: 0s - loss: 0.7952 - acc: 0.6669

INFO:tensorflow:Assets written to: ./res/multi_model_crossatt_pretrained.tf/assets


...

INFO:tensorflow:Assets written to: ./res/multi_model_crossatt_pretrained.tf/assets


86/86 [==============================] - 8s 93ms/step - loss: 0.7491 - acc: 0.6784 - val_loss: 0.9620 - val_acc: 0.5833
Epoch 4/30
80/86 [==========================>...] - ETA: 0s - loss: 0.7056 - acc: 0.6961

INFO:tensorflow:Assets written to: ./res/multi_model_crossatt_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_crossatt_pretrained.tf/assets


86/86 [==============================] - 9s 102ms/step - loss: 0.7044 - acc: 0.6988 - val_loss: 0.9265 - val_acc: 0.6140
Epoch 5/30
86/86 [==============================] - 1s 16ms/step - loss: 0.6756 - acc: 0.7200 - val_loss: 0.9943 - val_acc: 0.6272
Epoch 6/30
86/86 [==============================] - 1s 16ms/step - loss: 0.6391 - acc: 0.7361 - val_loss: 0.9683 - val_acc: 0.6118
Epoch 7/30
86/86 [==============================] - 1s 16ms/step - loss: 0.6083 - acc: 0.7420 - val_loss: 1.0022 - val_acc: 0.6140
Epoch 8/30
86/86 [==============================] - 1s 16ms/step - loss: 0.6093 - acc: 0.7420 - val_loss: 1.0570 - val_acc: 0.6053
Epoch 9/30
86/86 [==============================] - 1s 16ms/step - loss: 0.5938 - acc: 0.7427 - val_loss: 1.0551 - val_acc: 0.6053
Epoch 10/30
86/86 [==============================] - 1s 10ms/step - loss: 0.5968 - acc: 0.7537 - val_loss: 1.0327 - val_acc: 0.6031
Epoch 11/30
86/86 [==============================] - 1s 11ms/step - loss: 0.5804 - acc: 0.76

In [17]:
model = tf.keras.models.load_model('./res/multi_model_crossatt_pretrained.tf/')
pred = model.predict([np.asarray(visual_clip['test']), np.asarray(acoustic['test']), np.asarray(bert_embs['test'])])
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 1s 9ms/step
              precision    recall  f1-score   support

         Neg     0.6717    0.8911    0.7660       248
         Pos     0.6320    0.5643    0.5962       140
         Neu     0.3333    0.0145    0.0278        69

    accuracy                         0.6586       457
   macro avg     0.5457    0.4900    0.4633       457
weighted avg     0.6085    0.6586    0.6025       457

[[221  27   0]
 [ 59  79   2]
 [ 49  19   1]]


In [18]:
# './res/V_model_CNNLSTM_clip.tf'
vis_ipt = Input((10, 512))
v_model = tf.keras.models.load_model('./res/V_model_CNNLSTM_clip.tf/')
v_model = Model(inputs=v_model.inputs, outputs=v_model.layers[-2].output)
for l in v_model.layers:
    l.trainable = False
vis_h = v_model(vis_ipt)

# './res/A_model_Att-BLSTM_wav2vec_v2.tf'
aud_ipt = Input((128,512))
a_model = tf.keras.models.load_model('./res/A_model_Att-BLSTM_wav2vec_v2.tf/')
a_model = Model(inputs=a_model.inputs, outputs=a_model.layers[-2].output)
for l in a_model.layers:
    l.trainable = False
aud_h = a_model(aud_ipt)

# './res/T_model_AttCNN_bert.tf'
tex_ipt = Input((36,768))
t_model = tf.keras.models.load_model('./res/T_model_AttCNN_bert.tf/')
t_model = Model(inputs=t_model.inputs, outputs=[t_model.layers[-4].output,t_model.layers[-3].output])
for l in t_model.layers:
    l.trainable = False
tex_q, tex_qv_attention = t_model(tex_ipt)


v = Concatenate()([tex_q, tex_qv_attention])
q = Concatenate()([vis_h, aud_h])
print(v.shape, q.shape)
h = Attention(128)([v, q])
# h = Dense(64)(h)
res = Dense(3, activation='softmax')(h)

model = Model(inputs=[vis_ipt, aud_ipt, tex_ipt], outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

(None, 128) (None, 128)
Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 36, 768)]    0           []                               
                                                                                                  
 input_13 (InputLayer)          [(None, 10, 512)]    0           []                               
                                                                                                  
 input_14 (InputLayer)          [(None, 128, 512)]   0           []                               
                                                                                                  
 model_18 (Functional)          [(None, 64),         295040      ['input_15[0][0]']               
                                 (None, 64)]                       

In [19]:
callback_list = [ModelCheckpoint(filepath='./res/multi_model_crossattv2_pretrained.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=[np.asarray(visual_clip['train']), np.asarray(acoustic['train']), np.asarray(bert_embs['train'])], y=np.asarray(label['train']), batch_size=16, epochs=30, 
            validation_data=[[np.asarray(visual_clip['valid']), np.asarray(acoustic['valid']), np.asarray(bert_embs['valid'])], np.asarray(label['valid'])],
            callbacks=callback_list)

Epoch 1/30
85/86 [============================>.] - ETA: 0s - loss: 1.1111 - acc: 0.4257

INFO:tensorflow:Assets written to: ./res/multi_model_crossattv2_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_crossattv2_pretrained.tf/assets


86/86 [==============================] - 11s 105ms/step - loss: 1.1098 - acc: 0.4269 - val_loss: 0.9851 - val_acc: 0.5548
Epoch 2/30
86/86 [==============================] - ETA: 0s - loss: 0.9572 - acc: 0.5453

INFO:tensorflow:Assets written to: ./res/multi_model_crossattv2_pretrained.tf/assets


...

INFO:tensorflow:Assets written to: ./res/multi_model_crossattv2_pretrained.tf/assets


86/86 [==============================] - 7s 85ms/step - loss: 0.9424 - acc: 0.5541 - val_loss: 0.9786 - val_acc: 0.5614
Epoch 4/30
86/86 [==============================] - 1s 14ms/step - loss: 0.9469 - acc: 0.5556 - val_loss: 0.9811 - val_acc: 0.5548
Epoch 5/30
86/86 [==============================] - 1s 15ms/step - loss: 0.9452 - acc: 0.5731 - val_loss: 0.9817 - val_acc: 0.5482
Epoch 6/30
85/86 [============================>.] - ETA: 0s - loss: 0.9474 - acc: 0.5640

INFO:tensorflow:Assets written to: ./res/multi_model_crossattv2_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_crossattv2_pretrained.tf/assets


86/86 [==============================] - 9s 103ms/step - loss: 0.9465 - acc: 0.5651 - val_loss: 0.9777 - val_acc: 0.5482
Epoch 7/30
86/86 [==============================] - 1s 10ms/step - loss: 0.9328 - acc: 0.5592 - val_loss: 0.9795 - val_acc: 0.5482
Epoch 8/30
86/86 [==============================] - 1s 10ms/step - loss: 0.9213 - acc: 0.5892 - val_loss: 0.9815 - val_acc: 0.5482
Epoch 9/30
86/86 [==============================] - 1s 10ms/step - loss: 0.9377 - acc: 0.5607 - val_loss: 0.9839 - val_acc: 0.5417
Epoch 10/30
86/86 [==============================] - 1s 11ms/step - loss: 0.9265 - acc: 0.5658 - val_loss: 0.9865 - val_acc: 0.5482
Epoch 11/30
86/86 [==============================] - 1s 10ms/step - loss: 0.9210 - acc: 0.5738 - val_loss: 0.9906 - val_acc: 0.5439
Epoch 12/30
86/86 [==============================] - 1s 11ms/step - loss: 0.9313 - acc: 0.5746 - val_loss: 0.9920 - val_acc: 0.5439
Epoch 13/30
86/86 [==============================] - 1s 10ms/step - loss: 0.9227 - acc: 0.

In [20]:
model = tf.keras.models.load_model('./res/multi_model_crossattv2_pretrained.tf/')
pred = model.predict([np.asarray(visual_clip['test']), np.asarray(acoustic['test']), np.asarray(bert_embs['test'])])
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

         Neg     0.5754    0.9234    0.7090       248
         Pos     0.5085    0.2143    0.3015       140
         Neu     0.0000    0.0000    0.0000        69

    accuracy                         0.5667       457
   macro avg     0.3613    0.3792    0.3368       457
weighted avg     0.4680    0.5667    0.4771       457

[[229  19   0]
 [110  30   0]
 [ 59  10   0]]


/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/chason/anaconda3/envs/deepeye/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

# Model 4

In [21]:
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()
        
        
class CrossAttention(BaseAttention):
    def call(self, x, context):
        attn_output, attn_scores = self.mha(
            query=x,
            key=context,
            value=context,
            return_attention_scores=True)

        # Cache the attention scores for plotting later.
        self.last_attn_scores = attn_scores

        x = self.add([x, attn_output])
        x = self.layernorm(x)

        return x
    

class GlobalSelfAttention(BaseAttention):
    def call(self, x):
        attn_output = self.mha(
            query=x,
            value=x,
            key=x)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

In [28]:
# Multi_head Attention : Cross, Global

In [25]:
# './res/V_model_CNNLSTM_clip.tf'
vis_ipt = Input((10, 512))
v_model = tf.keras.models.load_model('./res/V_model_CNNLSTM_clip.tf/')
v_model = Model(inputs=v_model.inputs, outputs=v_model.layers[-2].output)
for l in v_model.layers:
    l.trainable = False
vis_h = v_model(vis_ipt)

# './res/A_model_Att-BLSTM_wav2vec_v2.tf'
aud_ipt = Input((128,512))
a_model = tf.keras.models.load_model('./res/A_model_Att-BLSTM_wav2vec_v2.tf/')
a_model = Model(inputs=a_model.inputs, outputs=a_model.layers[-2].output)
for l in a_model.layers:
    l.trainable = False
aud_h = a_model(aud_ipt)

# './res/T_model_AttCNN_bert.tf'
tex_ipt = Input((36,768))
t_model = tf.keras.models.load_model('./res/T_model_AttCNN_bert.tf/')
t_model = Model(inputs=t_model.inputs, outputs=[t_model.layers[-4].output,t_model.layers[-3].output])
for l in t_model.layers:
    l.trainable = False
tex_q, tex_qv_attention = t_model(tex_ipt)

v = Concatenate(axis=1)([tf.expand_dims(tex_q,1), tf.expand_dims(tex_qv_attention,1)])
q = Concatenate(axis=1)([tf.expand_dims(vis_h,1), tf.expand_dims(aud_h,1)])

# h = MultiHeadAttention(num_heads=4, key_dim=32, dropout=0.2)(v, q)
h = CrossAttention(num_heads=4, key_dim=32, dropout=0.2)(v, q)

# h = Dense(64)(h)
# res = Dense(3, activation='softmax')(h)
res = Dense(3, activation='softmax')(tf.reduce_mean(h, 1))

model = Model(inputs=[vis_ipt, aud_ipt, tex_ipt], outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_27"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 36, 768)]    0           []                               
                                                                                                  
 input_19 (InputLayer)          [(None, 10, 512)]    0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 128, 512)]   0           []                               
                                                                                                  
 model_26 (Functional)          [(None, 64),         295040      ['input_21[0][0]']               
                                 (None, 64)]                                               

In [26]:
callback_list = [ModelCheckpoint(filepath='./res/multi_model_multiheadatt_cross_pretrained.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=[np.asarray(visual_clip['train']), np.asarray(acoustic['train']), np.asarray(bert_embs['train'])], 
          y=np.asarray(label['train']), batch_size=16, epochs=30,
          validation_data=[[np.asarray(visual_clip['valid']), np.asarray(acoustic['valid']), np.asarray(bert_embs['valid'])],np.asarray(label['valid'])],
          callbacks=callback_list)

Epoch 1/30
115/115 [==============================] - ETA: 0s - loss: 0.7403 - acc: 0.6805

INFO:tensorflow:Assets written to: ./res/multi_model_multiheadatt_cross_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_multiheadatt_cross_pretrained.tf/assets


115/115 [==============================] - 11s 80ms/step - loss: 0.7403 - acc: 0.6805 - val_loss: 0.9297 - val_acc: 0.6053
Epoch 2/30
115/115 [==============================] - ETA: 0s - loss: 0.5828 - acc: 0.7627

INFO:tensorflow:Assets written to: ./res/multi_model_multiheadatt_cross_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_multiheadatt_cross_pretrained.tf/assets


115/115 [==============================] - 9s 81ms/step - loss: 0.5828 - acc: 0.7627 - val_loss: 0.9068 - val_acc: 0.6425
Epoch 3/30
115/115 [==============================] - 1s 13ms/step - loss: 0.5443 - acc: 0.7770 - val_loss: 0.9552 - val_acc: 0.6491
Epoch 4/30
115/115 [==============================] - 1s 10ms/step - loss: 0.5272 - acc: 0.7775 - val_loss: 0.9549 - val_acc: 0.6491
Epoch 5/30
115/115 [==============================] - 1s 9ms/step - loss: 0.5124 - acc: 0.7989 - val_loss: 0.9349 - val_acc: 0.6645
Epoch 6/30
115/115 [==============================] - 1s 10ms/step - loss: 0.4986 - acc: 0.8011 - val_loss: 0.9588 - val_acc: 0.6491
Epoch 7/30
115/115 [==============================] - 1s 9ms/step - loss: 0.4836 - acc: 0.8044 - val_loss: 0.9250 - val_acc: 0.6425
Epoch 8/30
115/115 [==============================] - 1s 10ms/step - loss: 0.4839 - acc: 0.8044 - val_loss: 0.9510 - val_acc: 0.6579
Epoch 9/30
115/115 [==============================] - 1s 10ms/step - loss: 0.4595 

In [27]:
model = tf.keras.models.load_model('./res/multi_model_multiheadatt_cross_pretrained.tf/')
pred = model.predict([np.asarray(visual_clip['test']), np.asarray(acoustic['test']), np.asarray(bert_embs['test'])])
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

         Neg     0.7276    0.8508    0.7844       248
         Pos     0.6838    0.5714    0.6226       140
         Neu     0.4000    0.2899    0.3361        69

    accuracy                         0.6805       457
   macro avg     0.6038    0.5707    0.5810       457
weighted avg     0.6647    0.6805    0.6671       457

[[211  22  15]
 [ 45  80  15]
 [ 34  15  20]]


In [32]:
# './res/V_model_CNNLSTM_clip.tf'
vis_ipt = Input((10, 512))
v_model = tf.keras.models.load_model('./res/V_model_CNNLSTM_clip.tf/')
v_model = Model(inputs=v_model.inputs, outputs=v_model.layers[-2].output)
for l in v_model.layers:
    l.trainable = False
vis_h = v_model(vis_ipt)

# './res/A_model_Att-BLSTM_wav2vec_v2.tf'
aud_ipt = Input((128,512))
a_model = tf.keras.models.load_model('./res/A_model_Att-BLSTM_wav2vec_v2.tf/')
a_model = Model(inputs=a_model.inputs, outputs=a_model.layers[-2].output)
for l in a_model.layers:
    l.trainable = False
aud_h = a_model(aud_ipt)

# './res/T_model_AttCNN_bert.tf'
tex_ipt = Input((36,768))
t_model = tf.keras.models.load_model('./res/T_model_AttCNN_bert.tf/')
t_model = Model(inputs=t_model.inputs, outputs=[t_model.layers[-4].output,t_model.layers[-3].output])
for l in t_model.layers:
    l.trainable = False
tex_q, tex_qv_attention = t_model(tex_ipt)

h = Concatenate(axis=1)([tf.expand_dims(tex_q,1), tf.expand_dims(tex_qv_attention,1), tf.expand_dims(vis_h,1), tf.expand_dims(aud_h,1)])

h = GlobalSelfAttention(num_heads=4, key_dim=32, dropout=0.2)(h)

# h = Dense(64)(h)
# res = Dense(3, activation='softmax')(h)
res = Dense(3, activation='softmax')(tf.reduce_mean(h, 1))

model = Model(inputs=[vis_ipt, aud_ipt, tex_ipt], outputs=res)
model.compile(optimizer='Adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics='acc')
model.summary()

Model: "model_35"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 36, 768)]    0           []                               
                                                                                                  
 input_25 (InputLayer)          [(None, 10, 512)]    0           []                               
                                                                                                  
 input_26 (InputLayer)          [(None, 128, 512)]   0           []                               
                                                                                                  
 model_34 (Functional)          [(None, 64),         295040      ['input_27[0][0]']               
                                 (None, 64)]                                               

In [33]:
callback_list = [ModelCheckpoint(filepath='./res/multi_model_multiheadatt_global_pretrained.tf', monitor='val_loss', save_best_only=True, save_freq='epoch')]

model.fit(x=[np.asarray(visual_clip['train']), np.asarray(acoustic['train']), np.asarray(bert_embs['train'])], 
          y=np.asarray(label['train']), batch_size=16, epochs=30,
          validation_data=[[np.asarray(visual_clip['valid']), np.asarray(acoustic['valid']), np.asarray(bert_embs['valid'])],np.asarray(label['valid'])],
          callbacks=callback_list)

Epoch 1/30
111/115 [===========================>..] - ETA: 0s - loss: 0.6876 - acc: 0.7072

INFO:tensorflow:Assets written to: ./res/multi_model_multiheadatt_global_pretrained.tf/assets


INFO:tensorflow:Assets written to: ./res/multi_model_multiheadatt_global_pretrained.tf/assets


115/115 [==============================] - 12s 77ms/step - loss: 0.6833 - acc: 0.7112 - val_loss: 0.9500 - val_acc: 0.6228
Epoch 2/30
115/115 [==============================] - 1s 13ms/step - loss: 0.5547 - acc: 0.7797 - val_loss: 0.9874 - val_acc: 0.6382
Epoch 3/30
115/115 [==============================] - 1s 12ms/step - loss: 0.5273 - acc: 0.7945 - val_loss: 0.9769 - val_acc: 0.6447
Epoch 4/30
115/115 [==============================] - 1s 11ms/step - loss: 0.5125 - acc: 0.8066 - val_loss: 0.9728 - val_acc: 0.6294
Epoch 5/30
115/115 [==============================] - 1s 10ms/step - loss: 0.4865 - acc: 0.8099 - val_loss: 1.0281 - val_acc: 0.6250
Epoch 6/30
115/115 [==============================] - 1s 11ms/step - loss: 0.4814 - acc: 0.8115 - val_loss: 0.9888 - val_acc: 0.6469
Epoch 7/30
115/115 [==============================] - 1s 12ms/step - loss: 0.4634 - acc: 0.8153 - val_loss: 1.0436 - val_acc: 0.6404
Epoch 8/30
115/115 [==============================] - 2s 15ms/step - loss: 0.46

In [34]:
model = tf.keras.models.load_model('./res/multi_model_multiheadatt_global_pretrained.tf/')
pred = model.predict([np.asarray(visual_clip['test']), np.asarray(acoustic['test']), np.asarray(bert_embs['test'])])
predicted_test_labels = pred.argmax(axis=1)
numeric_test_labels = np.array(label['test'])
            
eval_res = classification_report(numeric_test_labels, predicted_test_labels, 
                                    target_names = ['Neg', 'Pos', 'Neu'], 
                                    digits=4, output_dict=False)

print(eval_res)

cm = confusion_matrix(y_true=numeric_test_labels.tolist(), y_pred=predicted_test_labels.tolist())
print(cm)

15/15 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

         Neg     0.7529    0.7742    0.7634       248
         Pos     0.5965    0.7286    0.6559       140
         Neu     0.3871    0.1739    0.2400        69

    accuracy                         0.6696       457
   macro avg     0.5788    0.5589    0.5531       457
weighted avg     0.6498    0.6696    0.6515       457

[[192  48   8]
 [ 27 102  11]
 [ 36  21  12]]
